In [ ]:
import xarray as xr
import intake
import holoviews as hv
from holoviews.streams import Selection1D
import pandas as pd
import panel as pn
hv.extension('bokeh')

In [ ]:
master_catalog = 'https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml'

In [ ]:
ds = xr.open_dataset('great_lakes.nc')

In [ ]:
# gui = intake.gui
# gui.add(master_catalog)
# gui

In [ ]:
# ds = gui.item
# ds = ds.to_dask()

In [ ]:
ds_dims = [(k,v) for k,v in ds.dims.items()]
ds_coords = [coord for coord in ds.coords.keys()]
ds_data_vars = [var for var in ds.data_vars]
ds_attrs = [(k,v) for k,v in ds.attrs.items()]

## DataSet

In [ ]:
ds_coord_table = hv.Table(ds_coords,'Coords')
ds_coord_table.opts(width = 200);

### Dataset Dimension Table

In [ ]:
dim_table = hv.Table(ds_dims,['Dimension','Count'])
dim_table.opts(width = 200);

### DataSet Attributes

In [ ]:
attr_table = hv.Table(ds_attrs,['Attribute','Description'])
attr_table.opts(width = 500);

### DataSet Properties

In [ ]:
pn.Column(
    pn.Row(pn.pane.Markdown("### Dims"),pn.Spacer(width = 140),
           pn.pane.Markdown("### Coordinates"),pn.Spacer(width = 65),
           pn.pane.Markdown("### Attributes")
          ),
    pn.Row(dim_table,ds_coord_table,attr_table,)
)


## Variables

### Variable Name Table

In [ ]:
var_names = []
for var in ds.data_vars.values():
    var_names.append((var.name  ,
                      str([dict(var.attrs.items()).get('long_name')][0]),
                      ([coord for coord in var.coords.keys()])
                     ))

var_name_table = hv.Table(var_names,['Name','Long_Name','Coords'])
var_table_stream = Selection1D(source = var_name_table)
var_name_table.opts(width = 500,height = 300);

### Variable Attributes

In [ ]:
def display_attrs(index):
    if not index: 
        index =  0
    else:
        index = index[-1] 
    if var_name_table.data.empty:
        return hv.Table([],['Name','Long_Name','Coords'])
    else:
        var = var_name_table.iloc[index].data['Name'].values[0]

        v_attrs = [(k,v) for k,v in ds[var].attrs.items()]
        v_attrs_table = hv.Table(v_attrs,['Attribute','Description'])
        v_attrs_table.opts(width = 400,height = 300)
        return v_attrs_table

In [ ]:
dtable = hv.DynamicMap(display_attrs,streams=[var_table_stream])
# dtable

In [ ]:
h1 = pn.pane.Markdown("### Variables")
h2 = pn.pane.Markdown("### Attributes")

### Variable Properties

In [ ]:
pn.Column(pn.Row(h1,pn.Spacer(width = 400),h2),
          pn.Row(var_name_table,dtable)
         )